In [1]:
import pandas as pd
from sklearn import preprocessing
import altair as alt
# import networkx as nx
# import nx_altair as nxa
# from matplotlib.pyplot import figure
import numpy as np

In [19]:
df = pd.read_csv('/Users/adamoldenkamp/Documents/GitHub/Milestone-1-BGG/final_ranked.csv')
n = 30

def createCategoryHeatmap(df, top_pledge_percent, top_category_percent):
    top_category_percent = 100 - top_category_percent
    
    rank_df = df[df['ks_rank'].notna()].sort_values('usd_pledged', ascending=False)
    rank_df = rank_df.head(int(len(rank_df)*(top_pledge_percent/100)))

    category_cols = [x for x in rank_df.columns if 'category_' in x]
    df_category = rank_df[category_cols]
    df_category.columns = df_category.columns.str.replace('category_', '')
    df_category = df_category.fillna(0)

    adj = pd.DataFrame()

    for column in list(df_category.columns):
        t_df = pd.DataFrame(df_category[df_category[column] != 0].sum()).reset_index()
        adj = pd.concat([adj, t_df[0]], ignore_index=True, sort=False, axis=1)

    adj.columns = df_category.columns
    adj.index = df_category.columns
    np.fill_diagonal(adj.values, 0)

    keep = adj.sum().sort_values(ascending=True).to_frame()
    keep = list(keep.head(int(len(keep)*(top_category_percent/100))).index)

    adj = adj.drop(keep)
    adj = adj.drop(keep, axis=1)

    order = list(adj.sum().sort_values(ascending=True).index)
    
    adj = pd.melt(adj.reset_index(), id_vars=['index'])
    adj = adj[adj['value']!=0]
    
    chart = alt.Chart(adj).mark_rect().encode(
                alt.X('index:N', sort=order, title=None),
                alt.Y('variable:N', sort=order, title=None),
                alt.Color('value:Q',
                    scale=alt.Scale(scheme='greenblue'),
                    legend=alt.Legend(title='Total Records')
                )
            ).properties(width=550,height=550)
    
    text = alt.Chart(adj).mark_text(baseline='middle').encode(
                alt.X('index:N', sort=order),
                alt.Y('variable:N', sort=order),
                text='value:Q',
            )

    return chart + text

In [20]:
createCategoryHeatmap(df, 30, 20)

alt.LayerChart(...)